In [6]:
import pandas as pd
import numpy as np
import os 

#input for any file name
file_name=input("Enter File Name: ")
journey="up"
file_path=os.path.join('week_data',journey, file_name)

if not os.path.isfile(file_path):
    print(f"Error: File '{file_name}' does not exist.")
else:
    df=pd.read_csv(file_path)

Enter File Name: 04-02-19.csv


In [7]:
#Only 519 buses
bus_no = int(input("Enter bus Number: "))
df = df[df['Schedule Name'].str.contains(rf'\b{bus_no}\b')]

cols=['Schedule Name','Ticket Issued Time','Adult','From Stage','To Stage','Trip Start Time','Trip End Time','Source','Destination']
df=df[cols]

source_stage=input("Enter source: ").upper()
destination_stage=input("Enter destination: ").upper()
df=df[(df['Source']==source_stage) & (df['Destination']==destination_stage)]

# Convert the trip start time and end time columns to datetime objects
df['Ticket Issued Time'] = pd.to_datetime(df['Ticket Issued Time'], format='%H:%M:%S')

start=input("Enter Start time: ")
end=input("Enter End time: ")
start_time = pd.to_datetime(start).time()
end_time = pd.to_datetime(end).time()

df = df[(df['Ticket Issued Time'].dt.time >= start_time) & (df['Ticket Issued Time'].dt.time < end_time)]


Enter bus Number: 519
Enter source: t.nagar
Enter destination: thiruporur
				Boarding Time				
Enter Start time: 07:00:00
Enter End time: 08:00:00


In [8]:
bus_stages = [
    'T.NAGAR', 'SAIDAPET', 'ANNA UNIV', 'WPTC', 'SRP TOOLS',
    'KANDANCHAV', 'THORAIPAKKAM', 'M K CHAVADI', 'KARAPAKKAM',
    'SHOLINGANALLUR', 'KUMARAN NG', 'CHEMMANCHE', 'NAVALUR',
    'SIPCOT', 'CHURCH', 'PAL. CHEMI', 'HINDUSTAN', 'KELAMBAKKAM',
    'KOMAN NAGAR', 'ENGG', 'CHENGAMMAL', 'KALAVAKKAM','THIRUPORUR'
]

#assigning numbers from 0 to length of bus stages
stage_mapping = {stage: i for i, stage in enumerate(bus_stages)}

df['From Stage'] = df['From Stage'].map(stage_mapping)
df['To Stage'] = df['To Stage'].map(stage_mapping)
df.dropna(subset=['From Stage', 'To Stage'], inplace=True)


In [9]:
# Create an empty OD matrix with stage names as indexes and columns
od_matrix = pd.DataFrame(index=bus_stages, columns=bus_stages).fillna(0)

# Populate the matrix 
for index, row in df.iterrows():
    source = bus_stages[int(row['From Stage'])]
    destination = bus_stages[int(row['To Stage'])]
    adult_count = row['Adult']
    if not pd.isna(adult_count):
        od_matrix.loc[source, destination] += adult_count
    else:
        pass

od_matrix['Boarding'] = od_matrix.sum(axis=1)
od_matrix.loc['ALIGHTING', :] = od_matrix.sum(axis=0)

od_matrix

,T.NAGAR,SAIDAPET,ANNA UNIV,WPTC,SRP TOOLS,KANDANCHAV,THORAIPAKKAM,M K CHAVADI,KARAPAKKAM,SHOLINGANALLUR,...,CHURCH,PAL. CHEMI,HINDUSTAN,KELAMBAKKAM,KOMAN NAGAR,ENGG,CHENGAMMAL,KALAVAKKAM,THIRUPORUR,Boarding
T.NAGAR,0.0,0.0,0.0,2.0,5.0,5.0,0.0,4.0,8.0,6.0,...,9.0,2.0,6.0,3.0,0.0,0.0,0.0,2.0,1.0,64.0
SAIDAPET,0.0,0.0,1.0,1.0,5.0,4.0,5.0,2.0,0.0,1.0,...,1.0,1.0,5.0,3.0,0.0,0.0,0.0,0.0,0.0,35.0
ANNA UNIV,0.0,0.0,0.0,0.0,4.0,2.0,1.0,0.0,1.0,3.0,...,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,16.0
WPTC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,4.0,0.0,0.0,1.0,0.0,2.0,12.0
SRP TOOLS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,1.0,6.0
KANDANCHAV,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,4.0,1.0,0.0,0.0,0.0,2.0,3.0,12.0
THORAIPAKKAM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,6.0,11.0
M K CHAVADI,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,7.0
KARAPAKKAM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,8.0
SHOLINGANALLUR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,0.0,2.0,8.0,0.0,0.0,0.0,0.0,1.0,27.0


In [10]:
store_file=input("Enter store file ")
od_matrix.to_excel(f"{store_file}.xlsx")

KeyboardInterrupt: Interrupted by user